In [1]:
import os,sys
import numpy as np
from collections import namedtuple
import tqdm
import glob
import math
import random
import inspect
import os.path as osp
from pathlib import Path
import itertools
from itertools import chain
import numpy as np
import pandas as pd
import multiprocessing
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sys, os
from importlib import reload

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.data import Data, DataLoader, DataListLoader
from torch_geometric.nn import EdgeConv, global_mean_pool, DataParallel
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data,Dataset
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv
import h5py

In [103]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../')))

import models_torch.models as models
reload(models)
import models_torch.losses as losses
reload(losses)
import utils_torch.scaler
reload(utils_torch.scaler)
import utils_torch.preprocessing as prepr
import utils_torch.plot_util as plot
reload(plot)
import utils_torch.train_util as train
reload(train)
import graph_data.graph_data as graph_data
reload(graph_data)
import utils_torch.model_summary as summary
reload(summary)

sys.path.append(os.path.abspath(os.path.join('../../')))
import vande.util.util_plotting as vande_plot
reload(vande_plot)

sys.path.append(os.path.abspath(os.path.join('../../')))
sys.path.append(os.path.abspath(os.path.join('../../DarkFlow/darkflow/')))
import DarkFlow.darkflow.networks.flows as flows


In [3]:
data_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
dataset = graph_data.GraphDataset(root=data_dir,n_jets=1e3)
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
   
use_generator = False
if use_generator:
    validation_split = 0.2
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    if dataset_size > 2:
        split = int(np.floor(validation_split * dataset_size))
    else: 
        split = 1
    print(dataset_size,split)
    random_seed= 1001

    train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
                                                             generator=torch.Generator().manual_seed(random_seed))
    print("train subset dim:", len(train_subset))
    print("validation subset dim", len(val_subset))
    dataloaders = {
        'train':  DataLoader(train_subset, batch_size=128, shuffle=True),
        'val':   DataLoader(val_subset, batch_size=128, shuffle=True)
    }
    print("train_dataloader dim:", len(dataloaders['train']))
    print("val dataloader dim:", len(dataloaders['val']))
else : 
    in_memory_datas = dataset.return_inmemory_data() 

In [ ]:
pf_cands,jet_prop = dataset.pf_cands,dataset.jet_prop
pf_cands_t = torch.cat([torch.tensor(pf_cands[i], dtype=torch.float) for i in range(len(pf_cands))])
#Plot consistuents and jet features prepared for the graph! (but before any normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
jet_feats = 'N_constituents,M,Pt,Eta,Phi'.split(',')


save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
vande_plot.plot_features(pf_cands_t.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)
vande_plot.plot_features(jet_prop[:,0:-1], jet_feats ,'Normalized' , 'QCD', plotname='{}plot_jet_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [4]:
#scaler = standardize(train_subset) # I dont think that this works for the dataset implementation as it is done now, implement scaler as part of dataset ? 
scaler = prepr.standardize(in_memory_datas,minmax_idx=[3,4],log_idx=[3,4]) 

dataloaders = {
    'train':  DataLoader(in_memory_datas, batch_size=128, shuffle=True)
    }
print("train_dataloader dim:", len(dataloaders['train']))

#dataset.get(0).u

train_dataloader dim: 8


In [ ]:
pf_cands_norm = torch.cat([torch.tensor(in_memory_datas[i].x, dtype=torch.float) for i in range(len(in_memory_datas))])
#Plot consistuents and jet features prepared for the graph! (after normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats_normalized/'
vande_plot.plot_features(pf_cands_norm.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [5]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
multi_gpu = False #torch.cuda.device_count()>1

In [101]:
# model
input_dim = 7
output_dim = 4#7#4#4,7
big_dim = 32
hidden_dim = 2
#model = models.EdgeNet(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)
#model = models.EdgeNetVAE(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)
model = models.PlanarEdgeNetVAE(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = 10e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, threshold=1e-6)

model.to(device)
print(model)
summary.gnn_model_summary(model)

PlanarEdgeNetVAE(
  (mu_layer): Linear(in_features=32, out_features=2, bias=True)
  (var_layer): Linear(in_features=32, out_features=2, bias=True)
  (batchnorm): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  ))
  (decoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  ))
  (amor_u): Linear(in_features=32, out_features=12, bias=True)
  (amor_w): Linear(in_features=32, out_features=12, bias=True)
  (amor_b): Linear(in_features=32, out_features=6, bias=True)
  (flow_0): Planar(
    (h): Tanh()
    (softplus): Softplus(beta=1, threshold=20)
  )
  (flow_1): Planar(
    (h): Tanh()
  

In [104]:
#loss
#loss_ftn_obj = losses.LossFunction('mse_coordinates', device=device)
#loss_ftn_obj = losses.LossFunction('mse', device=device)
#loss_ftn_obj = losses.LossFunction('vae_loss_mse',beta=0.00001,device=device)
loss_ftn_obj = losses.LossFunction('vae_loss_mse_coord',beta=0.00001,device=device)

# Training loop
n_epochs = 80
stale_epochs = 0
loss = 999999
#train_losses = []
for epoch in range(0, n_epochs):
    #loss = train(model, optimizer, loader, len(datas), 128, loss_ftn_obj)
    loss_tot,loss_reco,loss_kl = train.train(model, optimizer, dataloaders['train'], len(dataloaders['train'].dataset), dataloaders['train'].batch_size, loss_ftn_obj)
    #train_losses.append(loss)
    print('Epoch: {:02d}, Training Loss Tot, Reco, KL :  {:.4f},{:.4f}, {:.4f}'.format(epoch, loss_tot,loss_reco,loss_kl))















  0%|          | 0/7.8125 [00:00<?, ?it/s]













train loss tot = 0.9598760,train loss reco = 0.9591177,train loss kl = 76.7933121:   0%|          | 0/7.8125 [00:00<?, ?it/s]













train loss tot = 0.9598760,train loss reco = 0.9591177,train loss kl = 76.7933121:   0%|          | 0/7.8125 [00:00<?, ?it/s]













train loss tot = 0.9598760,train loss reco = 0.9591177,train loss kl = 76.7933121:  13%|█▎        | 1/7.8125 [00:00<00:00,  7.08it/s]













train loss tot = 0.7227778,train loss reco = 0.7209011,train loss kl = 188.3990326:  13%|█▎        | 1/7.8125 [00:00<00:00,  7.08it/s]













train loss tot = 0.7227778,train loss reco = 0.7209011,train loss kl = 188.3990326:  13%|█▎        | 1/7.8125 [00:00<00:00,  7.08it/s]













train loss tot = 0.7227778,train loss reco = 0.7209011,train loss kl = 188.3990326:  26%|██▌       | 2/7.8125 [00:00<00:00,  7.70it/s]













train loss tot = 0.8720653,train loss reco = 0.8700520,train 

Epoch: 00, Training Loss Tot, Reco, KL :  0.7649,0.7632, 171.1530
















train loss tot = 0.6940501,train loss reco = 0.6919725,train loss kl = 208.4499664:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.94it/s]













train loss tot = 0.6940501,train loss reco = 0.6919725,train loss kl = 208.4499664:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.94it/s]













train loss tot = 0.6940501,train loss reco = 0.6919725,train loss kl = 208.4499664:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.82it/s]













train loss tot = 0.6286235,train loss reco = 0.6265020,train loss kl = 212.7740479:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.82it/s]













train loss tot = 0.6286235,train loss reco = 0.6265020,train loss kl = 212.7740479:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.82it/s]













train loss tot = 0.6286235,train loss reco = 0.6265020,train loss kl = 212.7740479:  38%|███▊      | 3/7.8125 [00:00<00:00,  9.79it/s]













train loss tot = 0.6211777,train loss reco = 0.6187739,train loss kl = 241.0022430:  38%|███▊     

Epoch: 01, Training Loss Tot, Reco, KL :  0.5954,0.5924, 299.6132
















train loss tot = 0.5075234,train loss reco = 0.4991918,train loss kl = 833.6625977:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.93it/s]













train loss tot = 0.5075234,train loss reco = 0.4991918,train loss kl = 833.6625977:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.93it/s]













train loss tot = 0.5075234,train loss reco = 0.4991918,train loss kl = 833.6625977:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.90it/s]













train loss tot = 0.5580287,train loss reco = 0.5479765,train loss kl = 1005.7702026:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.90it/s]













train loss tot = 0.5580287,train loss reco = 0.5479765,train loss kl = 1005.7702026:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.90it/s]













train loss tot = 0.5580287,train loss reco = 0.5479765,train loss kl = 1005.7702026:  38%|███▊      | 3/7.8125 [00:00<00:00,  9.89it/s]













train loss tot = 0.4769751,train loss reco = 0.4660714,train loss kl = 1090.8422852:  38%|███▊ 

...

Epoch: 77, Training Loss Tot, Reco, KL :  0.1606,0.1592, 139.1788
















train loss tot = 0.1482025,train loss reco = 0.1466643,train loss kl = 153.9692383:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.96it/s]













train loss tot = 0.1482025,train loss reco = 0.1466643,train loss kl = 153.9692383:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.96it/s]













train loss tot = 0.1482025,train loss reco = 0.1466643,train loss kl = 153.9692383:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.90it/s]













train loss tot = 0.2087794,train loss reco = 0.2072795,train loss kl = 150.1936035:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.90it/s]













train loss tot = 0.2087794,train loss reco = 0.2072795,train loss kl = 150.1936035:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.90it/s]













train loss tot = 0.2087794,train loss reco = 0.2072795,train loss kl = 150.1936035:  38%|███▊      | 3/7.8125 [00:00<00:00,  9.89it/s]













train loss tot = 0.1684258,train loss reco = 0.1669663,train loss kl = 146.1153870:  38%|███▊     

Epoch: 78, Training Loss Tot, Reco, KL :  0.1598,0.1583, 142.7249
















train loss tot = 0.1266265,train loss reco = 0.1252418,train loss kl = 138.5958557:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.86it/s]













train loss tot = 0.1266265,train loss reco = 0.1252418,train loss kl = 138.5958557:  13%|█▎        | 1/7.8125 [00:00<00:00,  9.86it/s]













train loss tot = 0.1266265,train loss reco = 0.1252418,train loss kl = 138.5958557:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.82it/s]













train loss tot = 0.1819535,train loss reco = 0.1806059,train loss kl = 134.9366150:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.82it/s]













train loss tot = 0.1819535,train loss reco = 0.1806059,train loss kl = 134.9366150:  26%|██▌       | 2/7.8125 [00:00<00:00,  9.82it/s]













train loss tot = 0.1819535,train loss reco = 0.1806059,train loss kl = 134.9366150:  38%|███▊      | 3/7.8125 [00:00<00:00,  9.78it/s]













train loss tot = 0.1651529,train loss reco = 0.1637443,train loss kl = 141.0232086:  38%|███▊     

Epoch: 79, Training Loss Tot, Reco, KL :  0.1604,0.1591, 137.7623


In [107]:
reload(utils_torch.scaler)
reload(prepr)
reload(plot)

save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
inverse_standardization = True
plot.plot_reco_for_loader(model, dataloaders['train'], device, scaler, inverse_standardization, 'test_train', osp.join(save_dir, 'reconstruction_post_train', 'train_reco_std_log_minmax_vae_planar_mseconv'), 'all_mseconv')


In [34]:
input_fts, reco_fts = plot.gen_in_out(model, dataloaders['train'], device)


4